<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/seminar_5_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estimating $P(C_k)$

Suppose in a training set there are 15 documents labeled $C_1="Physics"$, 25 documents labeled $C_2="Economics"$ and $C_3="Religion"$.

Estimate $P(C_k)$ using MLE (emphirical frequencies).

#Multinomial model parameters estimation

Suppose your dictionary contains words $\{"position", "velocity", "stocks"\}$.
Suppose you have $3$ documents labeled as "Physics".

Your 1st document contains 

* word "position" $x_1=0$ times
* word "velocity" $x_2=1$ times
* word "stocks" $x_3=0$ times.

Your 2nd document contains 

* word "position" $x_1=0$ times
* word "velocity" $x_2=1$ times
* word "stocks" $x_3=0$ times.

Your 3rd document contains 

* word "position" $x_1=3$ times
* word "velocity" $x_2=0$ times
* word "stocks" $x_3=0$ times.

Suppose you adopt multinomial model for word occurence

$$
P(\{x_1,x_2,x_3\}|"Phycics")=\frac{(x_1+x_2+x_3)!}{x_1!x_2!x_3!} p_{1}^{x_1}p_{2}^{x_2}p_{3}^{x_3}
$$

With given data use emphirical frequency (with smoothing $\alpha=0.1$) to estimate model parameters $p_1,p_2,p_3$ and verify that $p_1+p_2+p_3=1$.

Note: aggregate our three documents to get 

* word "position" $x_1=3$ times
* word "velocity" $x_2=2$ times
* word "stocks" $x_3=0$ times.




##Solution:

We have $K=3$ features ($3$ words in the vocabulary) and $n=5$ trials.


$$
p_i=\frac{n_i+ \alpha}{N+K \alpha}.
$$

So

$$
\begin{aligned}
p_1&=\frac{n_1+ \alpha}{n+K \alpha}=\frac{3+ 0.1}{5+3 \times 0.1}=0.58, \\
p_2&=\frac{n_2+ \alpha}{n+K \alpha}=\frac{2+ 0.1}{5+3 \times 0.1}=0.4, \\
p_3&=\frac{n_3+ \alpha}{n+K \alpha}=\frac{0+ 0.1}{5+3 \times 0.1}=0.02.
\end{aligned}
$$

We see that

$$
p_1+p_2+p_3=0.58+0.4+0.02=1.
$$



#Multivariate Bernoulli model parameters estimation

In the previos setup adopt multivariate Bernoulli model. 
Now we need to count not total number of occurencies in all documents but rather number of documents in which the word occurs.

* word "position" is present in $2$ documents
* word "velocity" is present in $2$ documents
* word "stocks" $x_3=0$ times.

The model 

$$
P(\{x_1,x_2,x_3\})= p_{1}^{x_1}(1-p_1)^{1-x_1} \times p_{2}^{x_2}(1-p_2)^{1-x_2} \times p_{3}^{x_3}(1-p_3)^{1-x_3}
$$

where $x_i$ is either $0$ or $1$ (i.e. the word is either present in the document or not).

With given data use emphirical frequency (with smoothing $\alpha=0.1$) to estimate model parameters $p_1,p_2,p_3$ and verify that $p_1+p_2+p_3=1$.



##Solution

$$
\begin{aligned}
p_1&=\frac{n_1+ \alpha}{n+K \alpha}=\frac{2+ 0.1}{4+3 \times 0.1}=0.49, \\
p_2&=\frac{n_2+ \alpha}{n+K \alpha}=\frac{2+ 0.1}{4+3 \times 0.1}=0.49, \\
p_3&=\frac{n_3+ \alpha}{n+K \alpha}=\frac{0+ 0.1}{4+3 \times 0.1}=0.02.
\end{aligned}
$$
